## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541811/question-answering-with-youtube-videos-using-rag-in-langchain

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Importing and Installing Required Libraries

In [1]:
!pip install -qU langchain 
!pip install -qU langchain-community 
!pip install -qU langchain-openai 
!pip install -qU youtube-transcript-api 
!pip install -qU pytube 
!pip install -qU chromadb
!pip install -qU faiss-cpu

In [30]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
import os

## Creating Text Documents from YouTube Videos

In [58]:
urls = [
    "https://www.youtube.com/watch?v=C5BkxbbLbIY",
    "https://www.youtube.com/watch?v=7qbJvucsU-Y",
    "https://www.youtube.com/watch?v=DNNMS7l6A-g",
    "https://www.youtube.com/watch?v=1rh-P2N0DR4",
    "https://www.youtube.com/watch?v=AW2kppo3rtI",
    "https://www.youtube.com/watch?v=lznP52OEpQk",
    "https://www.youtube.com/watch?v=ul14Qs24-s8",
    "https://www.youtube.com/watch?v=sB3N0ZqjPyM",
    "https://www.youtube.com/watch?v=JG1zkvwrn3E"
]
docs = []
for url in urls:
    docs.extend(YoutubeLoader.from_youtube_url(url, add_video_info=True).load())
    
    
print("=====================================")
print(f"Total number of videos: {len(docs)}")
print("=====================================")
print("Contents of video 1")
print("=====================================")
print(docs[0])

Total number of videos: 9
Contents of video 1
page_content="Paris the City of Lights love and endless magic this magical City always has something fun to do thanks to its long past famous landmarks and Lively culture Paris has something for everyone you can take a walk along the scene eat delicious sweets or look at worldclass art at the Louver but there is so much to see and do how do you decide what to do first don't worry viewers we've put together the best list of the 10 best things to do in Paris so let's talk about them one by one 10 Mont Marr Mont Marr is a cute area on a hill with great views of Paris well-known artists who lived and worked in and around Mont Marr like to hang out there Mont Marr's pretty streets and interesting people gave artists like Picasso and Van GA ideas you can still see paintings and other artists at work on their next projects the famous sakre cord church is at the top of the hill it is an important political and cultural landmark and a work of archit

In [43]:
type(docs[0])

langchain_core.documents.base.Document

## Splitting and Embedding YouTube Video Documents

In [44]:
openai_key = os.environ.get('OPENAI_KEY2')

In [45]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [46]:
embeddings = OpenAIEmbeddings(openai_api_key = openai_key)
vector = FAISS.from_documents(documents, embeddings)

## Question Answering with YouTube Videos

In [47]:
llm = ChatOpenAI(
    openai_api_key = openai_key ,
    model = 'gpt-4',
    temperature = 0.5
)

retriever = vector.as_retriever()

In [48]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

history_retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [49]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])
document_chain = create_stuff_documents_chain(llm, prompt)

In [50]:
retrieval_chain = create_retrieval_chain(history_retriever_chain, document_chain)

In [51]:
result = retrieval_chain.invoke({
    "chat_history": [],
    "input": "What are the top 3 things to do?"
})

print(result['answer'])

The top three things to do based on the context are: 

1. Visiting the top of the Ark (Arc de Triomphe) for a view of the city.
2. Having a picnic near the Eiffel Tower with food from a local market.
3. Renting bikes and riding around the city with no specific destination.


## A Command Line YouTube Question Answering Chatbot

In [52]:
chat_history = []

def generate_response_with_memory(query):
    result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": query
    })

    response = result['answer']
    chat_history.extend([HumanMessage(content = query),
                       AIMessage(content = response)])
    
    return response

In [55]:
print("=======================================================================")
print("A Chatbot that tells you things to do in Paris. Enter your query")
print("=======================================================================")

query = ""
while query != "bye":
    query = input("\033[1m User >>:\033[0m")
    
    if query == "bye":
        chat_history = []
        print("\033[1m Chatbot>>:\033[0m Thank you for your messages, have a good day!")
        break
    response = generate_response_with_memory(query)
    print(f"\033[1m Chatbot>>:\033[0m {response}")


A Chatbot that tells you things to do in Paris. Enter your query
 User >>:What are the absolute top 3 things to do?
 Chatbot>>: Based on the context, the top three things to do would be to visit the Arc de Triomphe and go to the top for an amazing view of the city, have a picnic near the Eiffel Tower, and rent bikes to ride around the city with no specific destination.
 User >>:what about the food?
 Chatbot>>: The context mentions several French dishes that you should try when you're in Paris. These include frog legs, which are described as soft and tender with a lot of garlic, a dish called "coco van," which is chicken in a red wine sauce stewed for a long time with onions, bacon, and mushrooms, and a dessert called "creme brulee," which is a cream dessert with a caramelized sugar top. There's also mention of French pastries and baked goods, specifically macarons and eclairs, which are highly recommended.
 User >>:What are some water activities?
 Chatbot>>: The context mentions taking